https://github.com/jpcolino/IPython_notebooks/blob/master/Least%20Square%20Monte%20Carlo%20Implementation%20in%20a%20Python%20Class.ipynb

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
def LSM(InitialValue, StrikePrice, InterestRate, Sigma, Maturity, Steps, Replications):
    option_type = 'put'
    Interval = Maturity/Steps
    discount= np.exp(-InterestRate*Interval)
    SIgma2 = Sigma*Sigma/2

    np.random.seed(1)
    PriceMatrix = np.zeros((Steps + 1, Replications), dtype=np.float64)
    PriceMatrix[0,:] = InitialValue
    for i in range(1, Steps + 1):
        Z = np.random.standard_normal( Replications//2)
        Z = np.concatenate((Z, -Z))
        PriceMatrix[i, :] = (PriceMatrix[i - 1, :]* np.exp((InterestRate - Sigma ** 2 / 2.) * Interval+ 
                                                           Sigma * Z * np.sqrt(Interval)))
    if option_type == 'call':
        Payoff = np.maximum(PriceMatrix - StrikePrice, np.zeros((Steps + 1, Replications),dtype=np.float64))
    else:
        Payoff = np.maximum(StrikePrice - PriceMatrix,np.zeros((Steps + 1, Replications),dtype=np.float64))

    value_matrix = np.zeros_like(Payoff)
    value_matrix[-1, :] = Payoff[-1, :]
    for t in range(Steps - 1, 0 , -1):
        regression = np.polyfit(PriceMatrix[t, :], value_matrix[t + 1, :] * discount, 2)
        continuation_value = np.polyval(regression, PriceMatrix[t, :])
        value_matrix[t, :] = np.where(Payoff[t, :] > continuation_value,Payoff[t, :],value_matrix[t + 1, :] * discount)

    value_vector = value_matrix[1,:]*discount

   # return np.sum(value_vector) / float(Replications), 
    return value_vector

In [3]:
def CI_95(data):
    a = np.array(data)
    n = len(a)
    m = np.mean(a)
    var = ((np.std(a))**2)*(n/(n-1))
    hw = 1.96*np.sqrt(var/n)
    print('Price of the Option is:',m)
    print('\nThe 95% CI is:',[m-hw,m+hw],'\n')
    return #m, [m-hw,m+hw]

In [4]:
CI_95(LSM(20, 23, 0.055, 0.50, 1, 252,15000))

Price of the Option is: 4.998438418391729

The 95% CI is: [4.931538075911112, 5.065338760872347] 



In [5]:
CI_95(LSM(15, 17, 0.0255, 0.350, 1, 252,15000))

Price of the Option is: 3.0170374526272545

The 95% CI is: [2.9778436712848944, 3.0562312339696147] 



In [6]:
CI_95(LSM(90, 100, 0.0255, 0.350, 1, 252,15000))

Price of the Option is: 16.755372980981825

The 95% CI is: [16.526693158557773, 16.984052803405877] 



In [7]:
CI_95(LSM(85, 95, 0.0325, 0.75, 1, 252,15000))

Price of the Option is: 27.92528064143853

The 95% CI is: [27.566047163924537, 28.284514118952522] 



https://github.com/alanklam/Course-Work/blob/master/AmericanOptionPricing/AmericanOptionLSM.py